In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import pickle

import sys
sys.path.append('../../')
from scripts.viz_tools import animate_patch
from scripts.dl_utils import download_patch, rect_from_point

In [ ]:
# This function takes a while to run as it is extracting data from GEE

rect_width = 0.0075
coords = [116.09064833699657, -8.64548776888652]
start_date = '2016-01-01'
end_date = '2021-05-01'
patches = download_patch(rect_from_point(coords, rect_width), start_date, end_date)
print(len(patches))

In [ ]:
name = "Lombok Kebon Kongkok"
file_name = f"../../figures/videos/{name} {coords[1]:.2f}, {coords[0]:.2f} - {rect_width}.mp4"
animate_patch(patches, file_name)